## Segmenting and Clustering Neighborhoods in Toronto

In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests

In [85]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"}
r = requests.get(url, headers = header)
table = pd.read_html(r.text)
df = pd.DataFrame(table[0])
df.columns=['Postcode','Borough','Neighbourhood']
df.drop([0], axis=0, inplace = True)

In [89]:
df.reset_index()
dfDropIndex= df[df['Borough']=='Not assigned'].index
df.drop(dfDropIndex, axis=0, inplace = True)

In [91]:
df1= df.groupby('Postcode').agg(lambda x:','.join(set(x)))

In [92]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.shape

(103, 2)

In [96]:
url2='http://beautiful-soup-4.readthedocs.io/en/latest/'
header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"}
r = requests.get(url2, headers = header)
table = pd.read_html(r.text)
df2 = pd.DataFrame(table[0])
df2.drop([0], axis=0, inplace = True)
df2

,0,1,2,3
1,Pythonâs html.parser,"BeautifulSoup(markup, ""html.parser"")",Batteries included Decent speed Lenient (As of...,"Not as fast as lxml, less lenient than html5lib."
2,lxmlâs HTML parser,"BeautifulSoup(markup, ""lxml"")",Very fast Lenient,External C dependency
3,lxmlâs XML parser,"BeautifulSoup(markup, ""lxml-xml"") BeautifulSou...",Very fast The only currently supported XML parser,External C dependency
4,html5lib,"BeautifulSoup(markup, ""html5lib"")",Extremely lenient Parses pages the same way a ...,Very slow External Python dependency
